<a href="https://colab.research.google.com/github/csaybar/EEwPython/blob/master/GLC30.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<!--COURSE_INFORMATION-->
<img align="left" style="padding-right:10px;" src="https://sitejerk.com/images/google-earth-logo-png-5.png" width=5% >
<img align="right" style="padding-left:10px;" src="https://colab.research.google.com/img/colab_favicon_256px.png" width=6% >


>> *This notebook is part of the free course [EEwPython](https://colab.research.google.com/github/csaybar/EEwPython/blob/master/index.ipynb); the content is available [on GitHub](https://github.com/csaybar/EEwPython)* and released under the [Apache 2.0 License](https://www.gnu.org/licenses/gpl-3.0.en.html). 

<!--NAVIGATION-->
 < [xXx](cmip5.ipynb) | [Contents](index.ipynb) | [XxX](CNN_demo.ipynb)>

<a href="https://colab.research.google.com/github/csaybar/EEwPython/blob/master/dnn_demo.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>

<center>
<h1>Google Earth Engine with Python </h1>
<h2> Download the GLC30 product:  Just available for Peru and Ecuador</h2>
</center>

```
FILE: GLC30.ipynb
AUTHOR: Cesar Aybar
EMAIL: csaybar@gmail.com
ORGANIZATION: n/a 
MODIFIED BY: n/a
CREATION DATE: 8 June 2019
LAST MOD DATE: N/A
DEPENDENCIES: earthengine-api, geopandas , folium
PURPOSE: Simple way for download the GLC30.
```


Land Cover information is fundamental for environmental change studies, land resource management, sustainable development, and many other societal benefits. In **Google Earth Engine** existing different coarse Land use/cover products (e.g. MCD12Q1-500m) that are not able to capture the most significant human impacts on land systems due to its spatial resolution. Therefore, this short post aims to introduce and explain step by step how to download the [GLC30](https://reader.elsevier.com/reader/sd/pii/S0924271614002275?token=BF89D29031101CDB800927F3DA0DA572A4BBBA74220D9178A24F13A5140E629638CFA580F464F61E1500662B55B90A76) a relative new Global Land Cover product at 30-meter spatial resolution.

<center>
<img src='http://www.globallandcover.com/home/images/vegas.jpg'>
</center>


## 1. Installing

Before coding do not forget install and load the following packages and remenber that you can communicate with the bash console prepending an ! to the code. 

In [0]:
!pip install geopandas #tensorflow 2.0
!pip install earthengine-api==0.1.175 #earthengine API

## 2. Authentification

This tutorial needs interacting with others Google services. For accomplish this task, it's necessary to authenticate (as yourself). The code below shows you how to do it.

- **Google Drive**

We will use Google Drive for save us results.

In [0]:
from google.colab import drive
drive.mount('/content/drive')  

- **Authenticate to Earth Engine**

In [0]:
!earthengine authenticate 

## 3. Testing the software setup

In [0]:
# Earth Engine Python API
import ee
ee.Initialize()

In [5]:
import folium
import geopandas as gpd

# Define the URL format used for Earth Engine generated map tiles.
EE_TILES = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'

print('Folium version: ' + folium.__version__)
print('Geopandas version: ' + gpd.__version__)

Folium version: 0.8.3
Geopandas version: 0.5.1


## 4. Display either ee.Image or ee.Feature


In [0]:
def Mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    :param center: Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = v["tile_fetcher"].url_format,
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz

## 5. Preparing Geometry Data (Geopandas to EE)

In [17]:
#download the geometry 
import urllib
url = 'https://github.com/csaybar/csaybar.github.io/raw/master/data/provinces.geojson'
urllib.request.urlretrieve(url, 'Peru.geojson')

('Peru.geojson', <http.client.HTTPMessage at 0x7fb88cc17e80>)

In [0]:
# Reading the file
peruprov = gpd.read_file('Peru.geojson')
lima = peruprov[peruprov.NOMBRE == 'LIMA'].to_json() # Selecting Lima
lima_coords = eval(lima)['features'][0]['geometry']['coordinates'][0] #getting just the coordinates

# ee.Geometry -> ee.Feature -> ee.FeatureCollection
eeLima = ee.FeatureCollection([ee.Feature(ee.Geometry.Polygon(lima_coords),{'Name':'Lima'})])

In [28]:
# display the geometry using folium
center_lima = eeLima.geometry().centroid().getInfo()['coordinates']
center_lima.reverse()

fig =eeLima.draw(**{'color': 'FF0000', 'strokeWidth': 5}).getMapId()
Mapdisplay(center_lima,{'Lima':fig})

## 6. Reading and clipping the GLC30 dataset

In [30]:
# Remove 0 and 255 values (no data)
def removevalues(img):
  value_0 = img.neq(0)
  value_255 = img.neq(255)
  return img.updateMask(value_0).updateMask(value_255).divide(10).int()

# Define a palette for the distinct land cover classes.
glc30Palette = [
  'ff02d0', # 1-Cultaved Lands
  '147749', # 2-Forest    
  'a9d05f', # 3-Grassland  
  '3eb370', # 4-Shrublands
  '7ecef4', # 5-Wetlands
  '00569a', # 6-Water bodies          
  '6f6f6f',  # 7-Tundra
  '911818', # 8-Artificial Surfaces
  'cacaca', # 9-Bareland    
  'f9f3c1', # 10-Permanent snow and ice  
]

# Load the dataset
glc30_lima = ee.ImageCollection('users/csaybar/GLC30PERU')\
               .map(removevalues).max().clip(eeLima)

vizparams = {'min':1,'max':10,'palette':glc30Palette}
Mapdisplay(center_lima,{'glc30_lima':glc30_lima.getMapId(vizparams)},zoom_start=10)

In [31]:
# Ecuador
Ecuador = ee.Geometry.Polygon([[-79.25,0.10],[-79.25,0.47],[-79.76,0.47],[-79.76,0.10], [-79.25,0.10]])
center_ecu = Ecuador.centroid().getInfo()['coordinates']
center_ecu.reverse()
# Load the dataset
glc30_ecuador = ee.ImageCollection('users/csaybar/GLC30PERU')\
               .map(removevalues).max().clip(Ecuador)

vizparams = {'min':1,'max':10,'palette':glc30Palette}
Mapdisplay(center_ecu,{'glc30':glc30_ecuador.getMapId(vizparams)},zoom_start=10)

## 6. Export the results of EE to Google Drive

In [0]:
# Export the image, specifying scale and region.
task = ee.batch.Export.image.toDrive(**{
    'image': glc30,
    'description': 'GLC30LIMA',
    'folder':'GLC30',
    'scale': 30,
    'region': eeLima.geometry().getInfo()['coordinates']
})
task.start()

In [34]:
import time 
while task.active():
  print('Polling for task (id: {}).'.format(task.id))
  time.sleep(5)

Polling for task (id: VRF5Q7EEMVFTWJGK3JPSEPBI).
Polling for task (id: VRF5Q7EEMVFTWJGK3JPSEPBI).
Polling for task (id: VRF5Q7EEMVFTWJGK3JPSEPBI).
Polling for task (id: VRF5Q7EEMVFTWJGK3JPSEPBI).
Polling for task (id: VRF5Q7EEMVFTWJGK3JPSEPBI).
Polling for task (id: VRF5Q7EEMVFTWJGK3JPSEPBI).
Polling for task (id: VRF5Q7EEMVFTWJGK3JPSEPBI).
Polling for task (id: VRF5Q7EEMVFTWJGK3JPSEPBI).
Polling for task (id: VRF5Q7EEMVFTWJGK3JPSEPBI).
Polling for task (id: VRF5Q7EEMVFTWJGK3JPSEPBI).
Polling for task (id: VRF5Q7EEMVFTWJGK3JPSEPBI).


In [0]:
# Export the image, specifying scale and region.
task = ee.batch.Export.image.toDrive(**{
    'image': glc30,
    'description': 'GLC30ECUADOR',
    'folder':'GLC30',
    'scale': 30,
    'region': Ecuador.getInfo()['coordinates']
})
task.start()